In [47]:
import os
import math
import pandas as pd
import time
import operator


In [2]:
os.listdir ("./data/created")

['NGS-2017-reg-wk1-6_speed&role.csv']

In [3]:
df = pd.read_csv("./data/video_footage-control.csv")
print(df.shape)
df.iloc[:,-1][1]

(37, 10)


'http://a.video.nfl.com//films/vodzilla/153512/Huber_58_yard_punt-AVzOd2dr-20181121_123514795_5000k.mp4'

In [4]:
df = pd.read_csv("./data/video_footage-injury.csv")
df.iloc[:,-1][25]


'http://a.video.nfl.com//films/vodzilla/153260/Bolden_runs_into_Colquitt_during_punt-LpPhshZz-20181119_162648901_5000k.mp4'

In [5]:
speed_and_role = pd.read_csv("./data/created/NGS-2017-reg-wk1-6_speed&role.csv", sep =";", decimal = ".", parse_dates = ['Time'])
speed_and_role.head(2)

,Season_Year,GameKey,PlayID,GSISID,Time,x,y,dis,o,dir,Event,Role,Duration,Max_number_event,previous_x,previous_y,SamePlayer_SamePlay,speed,next_speed,RollingMean
0,2017,399,1336,23534.0,2017-09-08 01:30:53.400,78.739998,29.820000,0.02,199.979996,168.630005,punt_play,P,205,6,NaN,NaN,NaN,0.316223,0.316223,NaN
1,2017,399,1336,23534.0,2017-09-08 01:30:53.500,78.750000,29.790001,0.03,199.009995,166.580002,NoEvent,P,205,6,78.739998,29.82,0.0,0.316223,0.360542,NaN


In [6]:
speed_and_role.isnull().sum()

Season_Year                0
GameKey                    0
PlayID                     0
GSISID                     0
Time                       0
x                          0
y                          0
dis                        0
o                          0
dir                        0
Event                      0
Role                       0
Duration                   0
Max_number_event           0
previous_x                 1
previous_y                 1
SamePlayer_SamePlay        1
speed                      0
next_speed             19714
RollingMean                4
dtype: int64

In [7]:
len(set(speed_and_role.Role))

46

In [8]:
set(speed_and_role.Role)

{'GL',
 'GLi',
 'GLo',
 'GR',
 'GRi',
 'GRo',
 'P',
 'PDL1',
 'PDL2',
 'PDL3',
 'PDL4',
 'PDL5',
 'PDL6',
 'PDM',
 'PDR1',
 'PDR2',
 'PDR3',
 'PDR4',
 'PDR5',
 'PDR6',
 'PFB',
 'PLG',
 'PLL',
 'PLL1',
 'PLL2',
 'PLL3',
 'PLM',
 'PLR',
 'PLR1',
 'PLR2',
 'PLR3',
 'PLS',
 'PLT',
 'PLW',
 'PPL',
 'PPR',
 'PR',
 'PRG',
 'PRT',
 'PRW',
 'VL',
 'VLi',
 'VLo',
 'VR',
 'VRi',
 'VRo'}

In [9]:
"""Role_Dictionnaire = {
 'GLi':'GL',
 'GLo':'GL',
  'GRi':'GR',
 'GRo':'GR',
 'PDL4':'PDL1',
 'PDL5':'PDL2',
 'PDL6':'PDL3',
 'PDR4':'PDR1',
 'PDR5':'PDR2',
 'PDR6':'PDR3',
 'PLL1':'PLL',
 'PLL2':'PLL',
 'PLL3':'PLL',
 'PLR1':'PLR',
 'PLR2':'PLR',
 'PLR3':'PLR',
 'VLi':'VL',
 'VLo':'VL',
 'VRi':'VR',
 'VRo':'VR'}
speed_and_role.replace({'Role': Role_Dictionnaire}, inplace=True)"""

"Role_Dictionnaire = {\n 'GLi':'GL',\n 'GLo':'GL',\n  'GRi':'GR',\n 'GRo':'GR',\n 'PDL4':'PDL1',\n 'PDL5':'PDL2',\n 'PDL6':'PDL3',\n 'PDR4':'PDR1',\n 'PDR5':'PDR2',\n 'PDR6':'PDR3',\n 'PLL1':'PLL',\n 'PLL2':'PLL',\n 'PLL3':'PLL',\n 'PLR1':'PLR',\n 'PLR2':'PLR',\n 'PLR3':'PLR',\n 'VLi':'VL',\n 'VLo':'VL',\n 'VRi':'VR',\n 'VRo':'VR'}\nspeed_and_role.replace({'Role': Role_Dictionnaire}, inplace=True)"

In [10]:
len(set(speed_and_role.Role))

46

# Creation of plays' list

we want to put the columns in the most used order 

# Refaire le script de création de list_timeseries

In [230]:
list_timeseries = []
for game in set(speed_and_role.GameKey):
    sub_df = speed_and_role.loc[speed_and_role.GameKey==game]
    
    for play in set(sub_df.PlayID):
        sub_dataframe = sub_df.loc[sub_df.PlayID==play]
        sub_dict = {}
        for role in set(speed_and_role.Role):
            sub_dict[role]=sub_dataframe.loc[sub_dataframe.Role==role]
        list_timeseries.append(sub_dict)

In [231]:
len(list_timeseries)

903

In [232]:
"""Let's initiate the count"""
most_used_role = {}
for role in set(speed_and_role.Role):
    most_used_role[role]=0

In [ ]:
def return_array(x):
    return [x['speed'],x['dir']]
    
time1 = time.time()
play_timeseries = []
for play in list_timeseries: 
    first = True
    return_dataframe = pd.DataFrame()
    count = 0
    #print("Play:{}".format(play))
    for role in play :
        role_df = pd.DataFrame()
        sel_columns = ['Season_Year','GameKey','PlayID','Time','Event','Duration']
        
        for col in sel_columns:
            role_df[col]=play[role][col]
            
        #print("Play : {}".format(play[role].shape))
        
        if play[role].shape[0]==0: #If this position does not exist for the play
            count+=1
            role_df['Target_'+str(role)] = None
            #print("This step is skipped")
            #continue #We skip this step
        else :   
            role_df['Target_'+str(role)] = play[role].apply(return_array, axis = 1)#[play[role]['speed'], play[role]['dir']]
            most_used_role[role]+=1
            #print("Role : {}".format(role_df.shape))
        
        if first:
            return_dataframe = role_df
            first = False
            #print("Return {}".format(return_dataframe.shape))
        else : 
            #print(role)
            return_dataframe = pd.merge(return_dataframe, role_df, on =['Season_Year','GameKey','PlayID','Time','Duration','Event'],suffixes= ('','_'+str(role)), how = 'outer')
            #print("Return {}".format(return_dataframe.shape))
            #print(return_dataframe.columns)
        #print(role)
            
    play_timeseries.append(return_dataframe.copy())
            
print((time.time()-time1)/60)       
        
        
        

On veut maintenant fusionner les colonnes pour avoir toutes les infos su

In [132]:
play_timeseries[4].columns

Index(['Season_Year', 'GameKey', 'PlayID', 'Time', 'Event', 'Duration',
       'Target_GR', 'Target_GL', 'Target_PDR6', 'Target_PFB', 'Target_VL',
       'Target_PLR3', 'Target_PPL', 'Target_PLG', 'Target_VLo', 'Target_PLR1',
       'Target_PLM', 'Target_PLR', 'Target_PRW', 'Target_P', 'Target_PPR',
       'Target_PLL2', 'Target_PLW', 'Target_GRi', 'Target_PDR2', 'Target_VLi',
       'Target_PDR3', 'Target_PDL3', 'Target_PDL6', 'Target_GLi',
       'Target_PLR2', 'Target_PDL1', 'Target_VRi', 'Target_PDR5', 'Target_PLT',
       'Target_GLo', 'Target_PRG', 'Target_PDL4', 'Target_PRT', 'Target_PR',
       'Target_PDR4', 'Target_PDL5', 'Target_PLL', 'Target_PLL3',
       'Target_PDR1', 'Target_GRo', 'Target_PLS', 'Target_VRo', 'Target_PLL1',
       'Target_PDM', 'Target_VR', 'Target_PDL2'],
      dtype='object')

df = play_timeseries[5].copy()
min_date = df["Time"].min()
max_date = df["Time"].max()
print(min_date, max_date)
date_rng = pd.date_range(min_date, max_date, freq="100ms")

# Missing Values 

We want to fillNan with [0,0] but it's quite difficult  because you're not really meant to store nonscalar values in dataframe cells. For now it is not supported. 

In [117]:
play_timeseries[4].loc[:,play_timeseries[4].isnull().sum()==0].isnull().sum().count()

28

In [149]:
def null_array(x):
    return [0,0]
time1 = time.time()
count_nan = 0
for i,ts in enumerate(play_timeseries):
    for col in ts.loc[:,ts.isnull().sum()==ts.isnull().sum().max()].columns:
        ts[col] = ts[col].apply(null_array)
    count_nan += ts.isnull().sum().sum()
    if ts.isnull().sum().sum() !=0:
        print(i)
print((time.time()-time1)/60)
print("Verification test, number of Nans in all dataframes :{}".format(count_nan))

32
37
56
86
95
172
177
195
215
231
249
263
266
276
279
290
318
344
352
353
365
385
407
408
415
444
459
472
482
485
490
502
507
516
517
520
540
551
555
561
565
567
588
600
619
622
623
630
640
645
647
648
654
662
676
678
694
701
726
735
739
764
796
823
825
826
831
881
891
899
0.27061785062154137
Verification test, number of Nans in all dataframes :1617


In [150]:
play_timeseries[32].loc[:,play_timeseries[32].isnull().sum()==play_timeseries[32].isnull().sum().max()]

,Target_GR,Target_GL,Target_PLG,Target_VLo,Target_PLR,Target_PRW,Target_P,Target_PPR,Target_PLW,Target_PDR2,...,Target_PDL3,Target_PDL1,Target_PLT,Target_PRG,Target_PRT,Target_PR,Target_PDR1,Target_PLS,Target_VR,Target_PDL2
0,"[0.3000640869139204, 248.44999694824216]","[0.1413841379304182, 238.13999938964844]","[0.6082933113211284, 17.149999618530273]","[3.195339815132676, 18.8700008392334]","[0.1414380807498827, 74.37999725341797]","[0.14138413793014196, 56.08000183105469]","[0.0, 232.63999938964844]","[0.5656848856302794, 51.22000122070313]","[0.3162169092556123, 95.93000030517578]","[0.10000228881843044, 357.1300048828125]",...,"[0.14143808074975708, 0.6899999976158142]","[0.4999847412254898, 307.8399963378906]","[0.0, 107.91999816894531]","[0.0, 39.2400016784668]","[0.5099323251945801, 103.2699966430664]","[1.2083069967666438, 25.569999694824208]","[0.14138413793021729, 243.5399932861328]","[0.3605422212623332, 22.489999771118164]","[0.4472238314068373, 105.9800033569336]","[0.1000213623046875, 140.9600067138672]"
1,"[0.3000640869139204, 251.5]","[0.1413841379304182, 238.88999938964844]","[0.6082933113211284, 16.670000076293945]","[3.195339815132676, 19.200000762939453]","[0.1414380807498827, 104.51000213623048]","[0.14138413793014196, 50.86999893188477]","[0.0, 234.5800018310547]","[0.5656848856302794, 51.6599998474121]","[0.3162169092556123, 100.65000152587892]","[0.10000228881843044, 359.0299987792969]",...,"[0.14143808074975708, 351.239990234375]","[0.4999847412254898, 309.0299987792969]","[0.0, 110.13999938964844]","[0.0, 46.47999954223633]","[0.5099323251945801, 93.5500030517578]","[1.2083069967666438, 25.78000068664551]","[0.14138413793021729, 250.25999450683602]","[0.3605422212623332, 14.600000381469727]","[0.4472238314068373, 106.13999938964844]","[0.1000213623046875, 104.7699966430664]"
2,"[0.2235863285395524, 257.22000122070307]","[0.3606057024472844, 238.41999816894528]","[0.5999946594238281, 16.170000076293945]","[2.9529469399570645, 19.63999938964844]","[0.1000022888182528, 127.68000030517578]","[0.3162410359796361, 45.29999923706055]","[0.0, 238.6000061035156]","[0.5656444270676326, 52.18999862670898]","[0.4472238314068373, 105.44000244140624]","[0.1414380807497571, 359.07000732421875]",...,"[0.22357779666467054, 341.8800048828125]","[0.5385429749721988, 310.6000061035156]","[0.0, 111.83000183105469]","[0.1000213623046875, 51.25]","[0.4000091552734375, 83.33999633789062]","[1.1704609308026468, 25.76000022888184]","[0.141438080749732, 278.1199951171875]","[0.40000915527347297, 7.639999866485595]","[0.3605422198748383, 105.97000122070312]","[0.0, 61.97000122070313]"
3,"[0.2828222135338665, 255.7700042724609]","[1.1401446447363426, 237.2700042724609]","[0.4123061222505865, 15.649999618530273]","[2.7892688014617044, 20.06999969482421]","[0.20000457763671875, 139.83999633789062]","[0.3000068664550426, 40.43999862670898]","[0.10000228881832383, 246.9900054931641]","[0.5831150782134089, 52.22999954223633]","[0.3605475100536736, 108.13999938964844]","[0.14142459568765572, 1.8400000333786009]",...,"[0.2236375080734014, 333.0400085449219]","[0.7279988330628363, 313.5799865722656]","[0.14143808074983247, 113.9800033569336]","[0.0, 55.43999862670898]","[0.4471470624306808, 72.7699966430664]","[1.2083069967666729, 25.29999923706055]","[0.1414380807498827, 58.75]","[0.6324639776352603, 1.0499999523162842]","[0.5385465157115863, 106.48999786376952]","[0.0, 35.540000915527344]"
4,"[0.3605898357526116, 256.3099975585937]","[2.163332669691892, 237.1300048828125]","[0.1000022888183949, 13.920000076293945]","[2.4351608242788245, 20.5]","[0.412306122250621, 146.82000732421875]","[0.36056867251523217, 36.150001525878906]","[0.0, 262.45001220703125]","[0.6708016276374641, 52.06000137329102]","[0.3162108778620919, 109.7300033569336]","[0.0, 8.460000038146973]",...,"[0.22357779666468647, 325.9700012207031]","[0.8246122438395147, 316.19000244140625]","[0.0, 116.87999725341795]","[0.1000022888183949, 58.9099998474121]","[0.50001144409

In [151]:
"""Then finish the job on the remaining NaN values"""
for ts in play_timeseries:
    for col in ts.loc[:,ts.isnull().sum()>0].columns:
        for row in ts.loc[ts[col].isnull(), col].index:
            ts.at[row, col] = [0,0]

In [152]:
count_nan = 0
for ts in play_timeseries:
    count_nan += ts.isnull().sum().sum()
print(count_nan)

0


# Aggregation

In [153]:
sorted_x = sorted(most_used_role.items(), key=operator.itemgetter(1), reverse = True)

In [159]:
sorted_x[:5]

[('P', 3613), ('PRG', 3609), ('PLG', 3605), ('PLS', 3604), ('GL', 3597)]

In [167]:
#sorted_role = ['Season_Year','GameKey','PlayID','Time','Duration','Event']
sorted_role = []
for i in range(len(sorted_x)):
    sorted_role.append("Target_"+sorted_x[i][0])
print(sorted_role[:4])

['Target_P', 'Target_PRG', 'Target_PLG', 'Target_PLS']


In [168]:
test = play_timeseries[154].copy()

In [169]:
test.head(2)    

,Season_Year,GameKey,PlayID,Time,Event,Duration,Target_GR,Target_GL,Target_PDR6,Target_PFB,...,Target_PLL,Target_PLL3,Target_PDR1,Target_GRo,Target_PLS,Target_VRo,Target_PLL1,Target_PDM,Target_VR,Target_PDL2
0,2017,415,2764,2017-09-15 02:28:42.100,punt_play,196,"[0.4000091552734375, 182.92999267578125]","[0.0, 0.9399999976158142]","[0, 0]","[0, 0]",...,"[0, 0]","[0, 0]","[0.0, 92.01000213623048]","[0, 0]","[0.4000091552733665, 267.54000854492193]","[0, 0]","[0, 0]","[0, 0]","[0.0, 95.0199966430664]","[0.14141110676693328, 320.54998779296875]"
1,2017,415,2764,2017-09-15 02:28:42.200,NoEvent,196,"[0.4000091552734375, 184.0200042724609]","[0.0, 6.050000190734862]","[0, 0]","[0, 0]",...,"[0, 0]","[0, 0]","[0.0, 87.54000091552734]","[0, 0]","[0.4000091552733665, 275.1099853515625]","[0, 0]","[0, 0]","[0, 0]","[0.0, 96.33000183105469]","[0.14141110676693328, 326.70999145507807]"


In [176]:
test['Try'] = test[sorted_role].sum(axis = 1)
originated_list = ['Season_Year','GameKey','PlayID','Time','Duration','Event']

final_list = []
for i,ts in enumerate(play_timeseries):
    final_df = ts[originated_list].copy()
    final_df['Target'] = ts[sorted_role].sum(axis = 1)
    final_list.append(final_df)

In [197]:
df = final_list[0].copy()
df.loc[df.Event != "NoEvent"]

,Season_Year,GameKey,PlayID,Time,Duration,Event,Target
0,2017,399,3384,2017-09-08 03:13:46.500,357,punt_play,"[0.14143808074980732, 252.52999877929688, 0.59..."
112,2017,399,3384,2017-09-08 03:13:57.700,357,line_set,"[0.0, 314.9700012207032, 0.0, 65.7399978637695..."
141,2017,399,3384,2017-09-08 03:14:00.600,357,ball_snap,"[0.0, 285.17999267578125, 0.0, 118.05999755859..."
162,2017,399,3384,2017-09-08 03:14:02.700,357,punt,"[2.0999908447268467, 259.0599975585937, 1.2728..."
210,2017,399,3384,2017-09-08 03:14:07.500,357,punt_received,"[5.5082065878296005, 275.3699951171875, 8.0062..."
267,2017,399,3384,2017-09-08 03:14:13.200,357,out_of_bounds,"[4.804160476689288, 1.0099999904632568, 6.2817..."
356,2017,399,3384,2017-09-08 03:14:22.100,357,play_submit,"[1.0049707090273086, 288.57000732421875, 0.948..."


In [226]:
df = final_list[412].copy()
df.loc[df.Event != "NoEvent"]

,Season_Year,GameKey,PlayID,Time,Duration,Event,Target
506,2017,442,4525,2017-09-24 23:26:15.400,565,penalty_flag,"[1.5264424490977178, 238.8699951171875, 4.1761..."
507,2017,442,4525,2017-09-24 23:26:15.400,565,penalty_flag,"[1.5264424490977178, 238.8699951171875, 4.1761..."
884,2017,442,4525,2017-09-24 23:26:34.300,565,penalty_flag,"[2.7294746827112557, 192.67999267578125, 1.170..."
885,2017,442,4525,2017-09-24 23:26:34.300,565,penalty_flag,"[2.7294746827112557, 192.67999267578125, 1.170..."
944,2017,442,4525,2017-09-24 23:26:37.300,565,penalty_accepted,"[0.5099098818133411, 175.22000122070312, 1.140..."
945,2017,442,4525,2017-09-24 23:26:37.300,565,penalty_accepted,"[0.5099098818133411, 175.22000122070312, 1.140..."
1098,2017,442,4525,2017-09-24 23:26:45.000,565,play_submit,"[0.7279988330629045, 286.260009765625, 1.51321..."
1099,2017,442,4525,2017-09-24 23:26:45.000,565,play_submit,"[0.7279988330629045, 286.260009765625, 1.51321..."


Je vois des times qui sont doubles pour certains df le 323,412 notamment.
Sinon il y a une pénalité qui est sifflée visiblement.
Le 462 est étrange! le 532 aussi ! 

In [222]:
set(speed_and_role.Event)

{'NoEvent',
 'ball_snap',
 'fair_catch',
 'first_contact',
 'fumble',
 'fumble_defense_recovered',
 'fumble_offense_recovered',
 'huddle_break_offense',
 'huddle_start_offense',
 'kick_received',
 'lateral',
 'line_set',
 'man_in_motion',
 'out_of_bounds',
 'pass_arrived',
 'pass_forward',
 'pass_outcome_caught',
 'pass_outcome_incomplete',
 'pass_outcome_interception',
 'penalty_accepted',
 'penalty_declined',
 'penalty_flag',
 'play_submit',
 'punt',
 'punt_blocked',
 'punt_downed',
 'punt_fake',
 'punt_land',
 'punt_muffed',
 'punt_play',
 'punt_received',
 'run',
 'shift',
 'snap_direct',
 'tackle',
 'timeout',
 'timeout_tv',
 'touchback',
 'touchdown'}

In [228]:
count = 0
for i,ts in enumerate(final_list) :
    if ts.loc[(ts.Event == "ball_snap")].shape[0]==0:
        print("Df : {}, shape : {}".format(i,ts.shape))
        count+=1
        print(set(ts.Event))
        

Df : 102, shape : (495, 7)
{'NoEvent', 'penalty_flag', 'penalty_accepted', 'play_submit', 'line_set', 'punt_play'}
Df : 108, shape : (701, 7)
{'NoEvent', 'punt_play', 'penalty_flag', 'play_submit', 'line_set', 'penalty_declined'}
Df : 197, shape : (310, 7)
{'NoEvent', 'penalty_flag', 'penalty_accepted', 'play_submit', 'line_set', 'punt_play'}
Df : 323, shape : (514, 7)
{'NoEvent', 'snap_direct', 'first_contact', 'run', 'play_submit', 'tackle', 'line_set', 'punt_play'}
Df : 336, shape : (454, 7)
{'NoEvent', 'penalty_flag', 'play_submit', 'line_set', 'penalty_declined'}
Df : 354, shape : (326, 7)
{'NoEvent', 'penalty_flag', 'penalty_accepted', 'play_submit', 'line_set', 'punt_play'}
Df : 383, shape : (342, 7)
{'NoEvent', 'punt', 'play_submit', 'line_set', 'punt_play', 'fair_catch'}
Df : 412, shape : (1130, 7)
{'NoEvent', 'play_submit', 'penalty_flag', 'penalty_accepted'}
Df : 434, shape : (199, 7)
{'NoEvent', 'penalty_flag', 'penalty_accepted', 'play_submit', 'punt_play'}
Df : 462, shape

In [224]:
count #snap direct ne rajoute qu'un play.

23

In [175]:
test['Try'][0]

[0.0,
 23.84000015258789,
 0.0,
 217.75999450683602,
 0.19996643066413355,
 283.5199890136719,
 0.4000091552733665,
 267.54000854492193,
 0.0,
 0.9399999976158142,
 0.09994506835923288,
 142.0399932861328,
 0.10002136230475857,
 324.75,
 0.3162169092556123,
 231.02999877929688,
 0.4000091552734375,
 182.92999267578125,
 0.5098949190103723,
 70.52999877929688,
 0.0,
 260.42999267578125,
 0.0,
 92.01000213623048,
 0.10000228881832383,
 110.76000213623048,
 0.14141110676693328,
 320.54998779296875,
 0.0,
 180.2700042724609,
 0.1414380807498827,
 67.16999816894531,
 0,
 0,
 0.0,
 95.0199966430664,
 0.5385110971367161,
 199.5200042724609,
 0,
 0,
 0.412306122250621,
 189.2899932861328,
 0,
 0,
 0.0,
 57.68000030517578,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.6324700077919743,
 82.5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.0999832153319602,
 292.5899963378906,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

On ne peut pas vraiment centrer sur la fin parce que cela distingue trop les moments de tailles différentes. Ce qu'on veut c'est comparer les moments au vu du déroulement des actions après le punt. Je me demande même si on ne devrait pas normaliser pour pouvoir comparer. 

On devrait optimalement centrer sur le snapp et sur le moment moyen de snapping pour les 23 manquants. 

In [ ]:
time_maximum_before_snap = 0
for i, ts in enumerate(final_list):
    